In [53]:
import pandas as pd
import os

In [51]:
data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/raw_data/'
cleaned_data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/cleaned_data/'
variables_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/variables/'

In [89]:
xl = pd.ExcelFile(data_path+'Data_as_per_requested_till_31st_Aug_2023.xlsx')
res = len(xl.sheet_names) #Number of sheets

df = pd.read_excel(data_path+'Data_as_per_requested_till_31st_Aug_2023.xlsx',
                   sheet_name=0)

In [90]:
cols = df.columns.to_list()

In [91]:
cols_to_remove_for_aggregation = ['Report_Date', 'District_Id', 'Revenue_Name',
                                  'RC_Id', 'District_Name', 'DFC_Id', 'Block_Name',
                                 'Relief_Distribution_Center_Name', 'Child_friendly_Space',
                                 'Latitude', 'Longitude',
                                  'Relief_Distribution_Other_Item_Name', 'Relief_Distribution_Other_Item_Qty',
                                 'Relief_Distribution_Other_Item_Unit',
                                 'Human_Live_Lost_Type']

for col_to_remove in cols_to_remove_for_aggregation:
    try:
        cols.remove(col_to_remove)
    except:
        pass

In [92]:
grouped_df = df.groupby(['Report_Date', 'RC_Id'])[cols].sum()
grouped_df

Population_affected_Male  Population_affected_Female  \
Report_Date RC_Id                                                         
2021-05-15  2.0                         0.0                         0.0   
            3.0                         0.0                         0.0   
            4.0                         0.0                         0.0   
            10.0                        0.0                         0.0   
            16.0                        0.0                         0.0   
...                                     ...                         ...   
2023-08-30  175.0                       0.0                         0.0   
            176.0                       0.0                         0.0   
            177.0                       0.0                         0.0   
            178.0                       0.0                         0.0   
            188.0                    1127.0                      1095.0   

                   Population_affected_Total  House_Fully_Damaged_Kuccha  \
Report_Date RC_Id                                                          
2021-05-15  2.0                          0.0                         0.0   
            3.0                          0.0                         0.0   
            4.0                          0.0                         0.0   
            10.0                         0.0                         0.0   
            16.0                         0.0                         0.0   
...                                      ...                         ...   
2023-08-30  175.0                        0.0                         0.0   
            176.0                        0.0                         0.0   
            177.0                        0.0                         0.0   
            178.0                        0.0                         0.0   
            188.0                     2901.0                         0.0   

                   House_Fully_Damaged_Pukka  Total_House_Fully_Damaged  \
Report_Date RC_Id                                                         
2021-05-15  2.0                          0.0                        0.0   
            3.0                          0.0                        0.0   
            4.0                          0.0                        0.0   
            10.0                         0.0                        0.0   
            16.0                         0.0                        0.0   
...                                      ...                        ...   
2023-08-30  175.0                        0.0                        0.0   
            176.0                        0.0                        0.0   
            177.0                        0.0                        0.0   
            178.0                        0.0                        0.0   
            188.0                        0.0                        0.0   

                   House_Partially_Damaged_Kuccha  \
Report_Date RC_Id                                   
2021-05-15  2.0                               0.0   
            3.0                               0.0   
            4.0                               0.0   
            10.0                              0.0   
            16.0                              0.0   
...                                           ...   
2023-08-30  175.0                             0.0   
            176.0                             0.0   
            177.0                             0.0   
            178.0                             0.0   
            188.0                             0.0   

                   House_Partially_Damaged_Pukka  \
Report_Date RC_Id                                  
2021-05-15  2.0                              0.0   
            3.0                              0.0   
            4.0                              0.0   
            10.0                             0.0   
            16.0                             0.0   
...                     

In [93]:
variables = grouped_df.columns

In [75]:
for variable in variables:
    variable_df = grouped_df[variable].reset_index()
    variable_df['year'] = variable_df.Report_Date.dt.year
    variable_df['month'] = variable_df.Report_Date.dt.month
    variable_df['month'] = variable_df.month.map("{:02}".format)
    variable_df = variable_df.groupby(['year','month','RC_Id'])[variable].sum().reset_index()
    variable_df['year_month'] = variable_df['year'].astype(str) + '_' + variable_df['month'].astype(str)
    
    for year_month in variable_df['year_month'].unique():
        variable_df_monthly = variable_df[variable_df['year_month']==year_month]
        if os.path.exists(variables_path+variable):
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)
        else:
            os.mkdir(variables_path+variable)
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)